In [2]:
import tkinter as tk
from tkinter import *
from tkinter import messagebox
from PIL import Image, ImageTk

In [6]:
window = tk.Tk()
window.title("Project")
window.resizable(0,0)

load1 = Image.open("HDR.png")
photo1 = ImageTk.PhotoImage(load1)

header = tk.Button(window, image=photo1)
header.place(x=5, y = 0)

canvas1 = Canvas(window, width=500, height=250, bg="ivory")
canvas1.place(x = 5, y = 120)

l1 = tk.Label(canvas1, text="Digit", font=('Algerian', 20))
l1.place(x=5, y=0)

t1 = tk.Entry(canvas1, width=20, border=5)
t1.place(x=150, y = 7)

def screen_capture():
    import pyscreenshot as ImageGrab
    import time
    import os

    os.startfile("C:/Users/admin/AppData/Local/Microsoft/WindowsApps/Microsoft.Paint_8wekyb3d8bbwe/mspaint.exe")
    s1 = t1.get()
    os.chdir("D:/FPT/AI/Major4/Handwritten Digit Recognition/captured_images")
    os.mkdir(s1)
    os.chdir("D:/FPT/AI/Major4/Handwritten Digit Recognition/")

    images_folder = "captured_images/"+str(s1)+"/"
    time.sleep(15)
    for i in range(100,102):
        time.sleep(8)
        # PIL.ImageGrab.grab() method takes a snapshot of the screen. 
        # image from the screen starting at the coordinates (60, 170) on the screen and ending at the coordinates (400, 550)
        im = ImageGrab.grab(bbox=(60,170,400,550)) #x1, y1, x2, y2
        print("saved.......", i)
        # print(i, end=" ")
        # if(i%18==0):
        #     print()
        im.save(images_folder+str(i)+'.png')
        print("clear screen now and redraw now.......")
    messagebox.showinfo("Result","Capturing screen is completed!!")

b1 = tk.Button(canvas1, text="1. Open pain and capture the screen", font=('Algerian', 15), bg="orange", fg="black", command=screen_capture)
b1.place(x=5, y=50)

def generate_dataset():
    import cv2
    import csv
    import glob

    header = ["label"]
    for i in range(0, 784):
        header.append("pixel" + str(i))
    with open('GUI_dataset.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow(header)

    for label in range(10):
        dirList = glob.glob("captured_images/"+str(label)+"/*.png")

        for img_path in dirList:
            # cv2.imread() method loads an image from the specified file.
            im = cv2.imread(img_path)
            # cv2.cvtColor() method is used to convert an image from one color space to another
            im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
            # applying Gaussian blur to an image
            im_gray = cv2.GaussianBlur(im_gray,(15,15), 0)
            roi = cv2.resize(im_gray, (28,28), interpolation = cv2.INTER_AREA)

            data = []
            data.append(label)
            rows, cols = roi.shape

            ## Add pixel one by one into data array
            for i in range(rows):
                for j in range(cols):
                    k = roi[i,j]
                    if k > 100:
                        k = 1
                    else:
                        k = 0
                    data.append(k)
            with open('GUI_dataset.csv', 'a') as f:
                writer = csv.writer(f)
                writer.writerow(data)
    messagebox.showinfo("Result", "Generating dataset is complete!!")
    
b2 = tk.Button(canvas1, text="2. Generate dataset", font=('Algerian', 15), bg="pink", fg="blue", command=generate_dataset)
b2.place(x=5, y=100)

def train_save_accuracy():
    import pandas as pd
    from sklearn.utils import  shuffle
    data = pd.read_csv('GUI_dataset.csv')
    data = shuffle(data)
    X = data.drop(["label"], axis=1)
    Y = data["label"]
    from sklearn.model_selection import  train_test_split
    train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size = 0.2)
    # import joblib: Đây là một thư viện trong Python dùng để lưu trữ và tải các đối tượng Python, như các mô hình máy học đã được đào tạo.
    import joblib
    # SVC là một trong những thuật toán Support Vector Machine dùng để phân loại dữ liệu.
    from sklearn.svm import SVC
    #  sử dụng các hàm kernel để ánh xạ dữ liệu vào không gian cao chiều hơn để có thể tìm ra ranh giới phân chia tốt hơn trong không gian đó.
    classifier = SVC(kernel="linear", random_state = 6)
    classifier.fit(train_x, train_y)
    # thư viện Joblib trong Python để lưu trữ mô hình phân loại (classifier) 
    joblib.dump(classifier, "GUI_model/digit_recognizer")
    from sklearn import metrics
    prediction = classifier.predict(test_x)
    acc = metrics.accuracy_score(prediction, test_y)
    messagebox.showinfo("Result", f"Your accuracy is {acc}")

b3 = tk.Button(canvas1, text="3. Train the model, save it and calculate accurace", font=('Algerian', 15), bg="green", fg="white", command= train_save_accuracy)
b3.place(x=5, y=150)

def prediction():
    import joblib
    import cv2
    import numpy as np
    import time
    import pyscreenshot as ImageGrab
      
    model = joblib.load("GUI_model/digit_recognizer")
    
    img = ImageGrab.grab(bbox=(180, 550, 550, 750))
    img.save("paint.png")
    im = cv2.imread("paint.png")
    load = Image.open("paint.png")
    load = load.resize((280,280))
    photo = ImageTk.PhotoImage(load)

    # Lables can be text or images
    img = Label(canvas3, image=photo, width=280, height=280)
    img.image=photo
    img.place(x=0,y=0)

    im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    # applying Gaussian blur to an image
    im_gray = cv2.GaussianBlur(im_gray,(15,15), 0)

    # Threshold the image
    # cv2.threshold là một hàm trong thư viện OpenCV trong Python được sử dụng để áp dụng phép ngưỡng hóa 
    # trên một ảnh xám (grayscale image). Hàm này nhận đầu vào là ảnh xám (im_gray) và áp dụng một ngưỡng 
    # để chuyển đổi ảnh thành ảnh nhị phân (cv2.THRESH_BINARY).
    # 100: Đây là giá trị ngưỡng. Tất cả các giá trị điểm ảnh lớn hơn 100 sẽ được chuyển đổi thành 255 (trắng).
    # 255: Giá trị tối đa được sử dụng sau khi áp dụng ngưỡng. Trong trường hợp này, nếu giá trị điểm ảnh lớn hơn ngưỡng, chúng sẽ được gán giá trị này (trắng).
    # cv2.THRESH_BINARY: Đây là loại ngưỡng hóa, trong trường hợp này, nó sẽ chuyển đổi ảnh thành ảnh nhị phân, tức là chỉ có hai giá trị điểm ảnh là 0 hoặc 255 (đen hoặc trắng).
    # ret: Là giá trị ngưỡng được sử dụng trong quá trình ngưỡng hóa. Trong trường hợp này, giá trị 100.
    # im_th: Là ảnh sau khi đã được áp dụng ngưỡng, tức là ảnh nhị phân, chỉ có hai màu đen và trắng, dựa trên ngưỡng được chỉ định.

    ret, im_th = cv2.threshold(im_gray, 100, 255, cv2.THRESH_BINARY)
    roi = cv2.resize(im_th, (28,28), interpolation = cv2.INTER_AREA)
    rows, cols = roi.shape

    X = []

    ## Add pixel one bt one into data arry
    for i in range(rows):
        for j in range(cols):
            k = roi[i,j]
            if(k>100): k = 1
            else: k = 0
            X.append(k)
        
    predictions = model.predict([X])

    a1 = tk.Label(canvas3, text="Prediction= ", font=("Algerian",20))   
    a1.place(x= 5, y= 350)

    b1 = tk.Label(canvas3, text=predictions[0], font=("Algerian",20))
    b1.place(x= 200, y= 350)

b4 = tk.Button(canvas1, text="4. Live prediction", font=('Algerian', 15), bg="white", fg="red", command= prediction)
b4.place(x=5, y=200)


canvas2 = Canvas(window, width=500, height=270, bg="black")
canvas2.place(x = 5, y = 380)

def activate_paint(e):
    global lastx, lasty
    canvas2.bind('<B1-Motion>', paint)
    lastx, lasty = e.x, e.y

def paint(e):
    global lastx, lasty
    x,y  = e.x, e.y
    canvas2.create_line((lastx, lasty,x,y), width= 10, fill="white")
    lastx, lasty = x,y

# canvas2.bind() được sử dụng để bắt các sự kiện như click chuột, di chuột, nhấn phím, và nhiều sự kiện khác liên quan đến canvas.
canvas2.bind("<1>", activate_paint)

def clear():
    canvas2.delete("all")

btn = tk.Button(canvas2, text = "clear", fg="white",bg="green" , command = clear)
btn.place(x=0, y= 0)


canvas3 = Canvas(window, width=280, height=530, bg="green")
canvas3.place(x = 515, y = 120)

window.geometry("800x680")
window.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\admin\AppData\Local\Temp\ipykernel_7604\3678874756.py", line 125, in prediction
    import pyscreenshot as ImageGrab
ModuleNotFoundError: No module named 'pyscreenshot'
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\admin\AppData\Local\Temp\ipykernel_7604\3678874756.py", line 125, in prediction
    import pyscreenshot as ImageGrab
ModuleNotFoundError: No module named 'pyscreenshot'
